# Chapter 03 실습 2: 커스텀 콜백 구현

## 목표
에포크마다 현재 학습률을 기록하고 시각화하는 콜백을 구현한다.

## 실습 내용
- `tf.keras.callbacks.Callback`을 상속하여 `LearningRateLogger` 구현
- `ReduceLROnPlateau`와 함께 사용하여 학습률 변화 과정 관찰
- 학습률 변화 곡선과 손실 곡선을 함께 시각화
- 커스텀 콜백의 활용 패턴 학습

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print("TensorFlow 버전:", tf.__version__)

# 재현성을 위한 시드 고정
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [ ]:
# ---------------------------------------------------
# LearningRateLogger 커스텀 콜백 구현
# ---------------------------------------------------

class LearningRateLogger(tf.keras.callbacks.Callback):
    """에포크별 학습률을 기록하는 커스텀 콜백.
    
    ReduceLROnPlateau 등 학습률을 변경하는 콜백과 함께 사용하면
    학습률이 언제, 얼마나 변경되었는지 추적할 수 있다.
    
    Attributes:
        lr_history: 에포크별 학습률 기록 리스트
        epoch_history: 에포크 번호 리스트
    """
    
    def __init__(self, verbose=0):
        super().__init__()
        self.verbose = verbose
        self.lr_history = []        # 에포크별 학습률
        self.epoch_history = []     # 에포크 번호
        self.lr_change_epochs = []  # 학습률이 변경된 에포크
        self._prev_lr = None        # 이전 학습률 (변화 감지용)
    
    def on_train_begin(self, logs=None):
        """학습 시작 시 초기화"""
        self.lr_history = []
        self.epoch_history = []
        self.lr_change_epochs = []
        self._prev_lr = None
        
        initial_lr = float(self.model.optimizer.learning_rate)
        if self.verbose:
            print(f"LearningRateLogger 시작 - 초기 학습률: {initial_lr:.6f}")
    
    def on_epoch_end(self, epoch, logs=None):
        """에포크 종료 후 현재 학습률을 기록
        
        Args:
            epoch: 현재 에포크 번호 (0부터 시작)
            logs: 현재 에포크의 메트릭 딕셔너리
        """
        # 현재 학습률 가져오기
        current_lr = float(self.model.optimizer.learning_rate)
        
        # 기록
        self.lr_history.append(current_lr)
        self.epoch_history.append(epoch + 1)  # 1부터 시작
        
        # 학습률 변화 감지
        if self._prev_lr is not None and abs(current_lr - self._prev_lr) > 1e-10:
            self.lr_change_epochs.append(epoch + 1)
            if self.verbose:
                print(f"  [LR 변경] 에포크 {epoch+1}: "
                      f"{self._prev_lr:.6f} -> {current_lr:.6f}")
        
        self._prev_lr = current_lr
    
    def on_train_end(self, logs=None):
        """학습 완료 후 요약 출력"""
        if self.verbose:
            print(f"\n학습률 변경 횟수: {len(self.lr_change_epochs)}")
            if self.lr_change_epochs:
                print(f"변경 에포크: {self.lr_change_epochs}")
            print(f"초기 학습률: {self.lr_history[0]:.6f}")
            print(f"최종 학습률: {self.lr_history[-1]:.6f}")
    
    def get_lr_stats(self):
        """학습률 통계 반환"""
        return {
            'initial_lr': self.lr_history[0] if self.lr_history else None,
            'final_lr': self.lr_history[-1] if self.lr_history else None,
            'num_changes': len(self.lr_change_epochs),
            'change_epochs': self.lr_change_epochs,
            'min_lr': min(self.lr_history) if self.lr_history else None,
            'max_lr': max(self.lr_history) if self.lr_history else None,
        }


print("LearningRateLogger 콜백 정의 완료")
print("주요 메서드:")
print("  on_train_begin: 학습 시작 시 초기화")
print("  on_epoch_end: 매 에포크 후 학습률 기록 및 변화 감지")
print("  on_train_end: 학습 완료 후 요약")
print("  get_lr_stats: 학습률 통계 반환")

In [ ]:
# ---------------------------------------------------
# 모델 학습: ReduceLROnPlateau + LearningRateLogger
# ---------------------------------------------------

# 데이터 준비: MNIST
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train_full = X_train_full.reshape(-1, 784).astype('float32') / 255.0
X_test = X_test.reshape(-1, 784).astype('float32') / 255.0

X_train = X_train_full[:10000]
y_train = y_train_full[:10000]
X_val = X_train_full[10000:12000]
y_val = y_train_full[10000:12000]

# 모델 생성
tf.random.set_seed(SEED)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(784,)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

# 의도적으로 큰 학습률로 시작하여 ReduceLROnPlateau 동작 관찰
INITIAL_LR = 0.01
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=INITIAL_LR),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 콜백 설정
lr_logger = LearningRateLogger(verbose=1)  # 학습률 변화 출력 활성화

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,       # 학습률을 30%로 감소
    patience=4,       # 4 에포크 동안 개선 없으면 감소
    min_lr=1e-6,
    min_delta=0.005,
    verbose=1
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=0
)

# 학습 실행
print(f"\n초기 학습률: {INITIAL_LR}")
print(f"학습 시작...\n")

history = model.fit(
    X_train, y_train,
    epochs=40,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[
        lr_logger,          # 커스텀 콜백 (ReduceLROnPlateau보다 먼저 등록)
        reduce_lr,          # 학습률 자동 감소
        early_stopping      # 조기 종료
    ],
    verbose=0
)

# 결과 요약
lr_stats = lr_logger.get_lr_stats()
print(f"\n=== 학습 완료 요약 ===")
print(f"실제 학습 에포크 수: {len(history.history['loss'])}")
print(f"초기 학습률: {lr_stats['initial_lr']:.6f}")
print(f"최종 학습률: {lr_stats['final_lr']:.6f}")
print(f"학습률 변경 횟수: {lr_stats['num_changes']}")
print(f"학습률 변경 에포크: {lr_stats['change_epochs']}")
print(f"최고 검증 정확도: {max(history.history['val_accuracy']):.4f}")

In [ ]:
# ---------------------------------------------------
# 학습률 변화 곡선 시각화
# ---------------------------------------------------

n_epochs = len(history.history['loss'])
epochs_range = range(1, n_epochs + 1)

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. 학습률 변화 (로그 스케일)
ax1 = axes[0, 0]
ax1.semilogy(lr_logger.epoch_history, lr_logger.lr_history,
             'b-o', linewidth=2, markersize=4, label='학습률')

# 학습률 변경 시점 표시
for change_epoch in lr_logger.lr_change_epochs:
    ax1.axvline(x=change_epoch, color='red', linestyle='--',
                alpha=0.7, linewidth=1.5)

if lr_logger.lr_change_epochs:
    ax1.axvline(x=lr_logger.lr_change_epochs[0], color='red',
                linestyle='--', alpha=0.7, linewidth=1.5, label='학습률 감소 시점')

ax1.set_xlabel('에포크')
ax1.set_ylabel('학습률 (로그 스케일)')
ax1.set_title('학습률 변화 (LearningRateLogger)')
ax1.legend()
ax1.grid(True, alpha=0.3, which='both')

# 2. 훈련/검증 손실
ax2 = axes[0, 1]
ax2.plot(epochs_range, history.history['loss'],
         'b-', linewidth=2, label='훈련 손실')
ax2.plot(epochs_range, history.history['val_loss'],
         'r-', linewidth=2, label='검증 손실')

# 학습률 변경 시점 표시
for change_epoch in lr_logger.lr_change_epochs:
    ax2.axvline(x=change_epoch, color='orange', linestyle='--',
                alpha=0.8, linewidth=1.5)

ax2.set_xlabel('에포크')
ax2.set_ylabel('손실')
ax2.set_title('훈련/검증 손실 (주황 점선: LR 감소 시점)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. 훈련/검증 정확도
ax3 = axes[1, 0]
ax3.plot(epochs_range, history.history['accuracy'],
         'b-', linewidth=2, label='훈련 정확도')
ax3.plot(epochs_range, history.history['val_accuracy'],
         'r-', linewidth=2, label='검증 정확도')

for change_epoch in lr_logger.lr_change_epochs:
    ax3.axvline(x=change_epoch, color='orange', linestyle='--',
                alpha=0.8, linewidth=1.5)

ax3.set_xlabel('에포크')
ax3.set_ylabel('정확도')
ax3.set_title('훈련/검증 정확도 (주황 점선: LR 감소 시점)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. 학습률 변화 (선형 스케일, 세부 시각화)
ax4 = axes[1, 1]
ax4.plot(lr_logger.epoch_history, lr_logger.lr_history,
         'b-o', linewidth=2, markersize=5)
ax4.fill_between(lr_logger.epoch_history, lr_logger.lr_history,
                  alpha=0.2, color='blue')

# 각 변경 시점에 주석 추가
for i, change_epoch in enumerate(lr_logger.lr_change_epochs):
    idx = change_epoch - 1  # 0-based 인덱스
    if idx < len(lr_logger.lr_history):
        lr_val = lr_logger.lr_history[idx]
        ax4.annotate(
            f'에포크 {change_epoch}\n-> {lr_val:.5f}',
            xy=(change_epoch, lr_val),
            xytext=(change_epoch + 0.5, lr_val * 1.5),
            fontsize=8,
            arrowprops=dict(arrowstyle='->', color='red'),
            color='red'
        )

ax4.set_xlabel('에포크')
ax4.set_ylabel('학습률 (선형 스케일)')
ax4.set_title('학습률 변화 상세 (변경 시점 주석 포함)')
ax4.grid(True, alpha=0.3)

plt.suptitle('커스텀 LearningRateLogger + ReduceLROnPlateau 결과',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n=== 학습률 변화 기록 ===")
print(f"{'에포크':^8} {'학습률':^15} {'변화':^10}")
print("-" * 35)
prev_lr = None
for epoch, lr in zip(lr_logger.epoch_history, lr_logger.lr_history):
    changed = ""
    if prev_lr is not None and abs(lr - prev_lr) > 1e-10:
        changed = f"  <- {prev_lr:.6f} * 0.3"
    print(f"{epoch:^8d} {lr:^15.6f}{changed}")
    prev_lr = lr

## 도전 과제

### 도전 1: 조기 종료 커스텀 콜백 구현

아래 조건을 만족하는 `SmartEarlyStopping` 콜백을 구현해보세요:

- `patience` 에포크 동안 `val_accuracy`가 개선되지 않으면 학습 종료
- 학습 종료 시 "최고 성능 에포크"와 "최고 정확도"를 출력
- `min_delta` 파라미터로 개선의 최소 기준을 설정
- 학습 종료 시 `self.model.stop_training = True`로 학습을 중단

```python
class SmartEarlyStopping(tf.keras.callbacks.Callback):
    def __init__(self, patience=5, min_delta=0.001):
        super().__init__()
        self.patience = patience
        self.min_delta = min_delta
        # 여기에 초기화 코드 작성
    
    def on_epoch_end(self, epoch, logs=None):
        # 여기에 조기 종료 로직 구현
        pass
```

### 도전 2: 그래디언트 노름 모니터링 콜백

매 배치마다 그래디언트 노름(norm)을 기록하는 `GradientNormLogger` 콜백을 구현해보세요.
- `on_train_batch_end`를 사용하여 배치별 그래디언트 평균 노름을 기록
- 그래디언트 폭발(exploding gradients) 감지 기능 추가

### 도전 3: WarmupScheduler 구현

처음 `warmup_epochs` 동안 학습률을 선형으로 증가시킨 후 지정된 학습률로 유지하는 `WarmupScheduler` 콜백을 구현해보세요.

$$lr_{warmup}(e) = \frac{e}{warmup\_epochs} \cdot lr_{target}$$